In [1]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt

In [2]:
# Set up paths

path_to_data = '/expanse/nfs/cw3e/cwp140/'     # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## open precipitation and ivt dfs
## append precip to each community IVT df
option = 'a'
temporal_res = 'daily'
community_lst = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']
varname = 'PCPT' # 'PCPT' or 'UV'

fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format(varname)
wrf = xr.open_mfdataset(fname_pattern, combine='by_coords', engine='netcdf4')
if temporal_res == 'hourly':
    wrf = wrf
elif (temporal_res == 'daily') & (varname == 'PCPT'):
    wrf = wrf.resample(time="1D").sum('time') # resample WRF data to be mm per day
    
elif (temporal_res == 'daily') & (varname == 'UV'):
    wrf = wrf.sel(lev='1000')
    wrf = wrf.resample(time="1D").mean('time') # resample WRF data to be m s-1

wrf

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


<xarray.Dataset>
Dimensions:  (time: 14610, y: 250, x: 320)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 1980-01-02 ... 2019-12-31
    lat      (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    PCPT     (time, y, x) float32 dask.array<chunksize=(1, 250, 320), meta=np.ndarray>
Attributes:
    contact:  rtladerjr@alaska.edu
    info:     Alaska CASC
    data:     Downscaled CFSR
    format:   version 1
    date:     Sat Dec  8 15:40:16 AKST 2018

In [4]:
%%time
## need to rechunk so time is a single chunk
wrf = wrf.chunk(dict(time=-1))

quantile_arr = np.array([0, .75, .95])

## Calculate the percentiles
quantile = wrf.quantile(quantile_arr, dim=['time'], skipna=True)

CPU times: user 624 ms, sys: 1.88 ms, total: 626 ms
Wall time: 627 ms


In [5]:
quantile

<xarray.Dataset>
Dimensions:   (quantile: 3, y: 250, x: 320)
Coordinates:
    lat       (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
    lon       (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
  * quantile  (quantile) float64 0.0 0.75 0.95
Dimensions without coordinates: y, x
Data variables:
    PCPT      (quantile, y, x) float64 dask.array<chunksize=(3, 250, 320), meta=np.ndarray>

In [6]:
%%time
# write to netCDF
fname = os.path.join(path_to_data, 'preprocessed/SEAK-WRF-PCPT/PCPT_quantiles.nc')
quantile.load().to_netcdf(path=fname, mode = 'w', format='NETCDF4')

CPU times: user 1min 48s, sys: 1min 21s, total: 3min 10s
Wall time: 7min 30s
